In [1]:
import random
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize_scalar


In [362]:
random.seed(188)
n=10000
T=20
phizero=0.7
a=np.array([np.random.normal(0,1) for i in range(n)])
y0=np.array([np.random.normal(a[i]/(1-phizero),1/np.sqrt(1-phizero**2)) for i in range(n)])
epsilon=[np.random.normal(0,1) for i in range(n*T)]

In [363]:
y=np.zeros((T,n))
y[0]=y0
y=y.T
for i in range(n):
    for j in range(1,T):
        y[i][j]=a[i]+phizero*y[i][j-1]+epsilon[i*T+j]

yit=y.reshape(n*T)

In [364]:
#Generation des first difference :
delta_y=np.zeros((n,T-1))
for i in range(n):
    for j in range(T-1):
        delta_y[i,j]=y[i,j+1]-y[i,j]

1) Pour chaque $i$, on définit les instruments $z_{it}$ pertinents tel que : $\forall t>1, E(z_{it}\Delta u_{it})=0$

En l'occurence, on prend : $z_{it}=(y_{it-2}) \forall t\ge2$ 0 sinon (pas d'instruments). Comme on instrumente directement les différences premières, il n'y a pas de t=0.

$Z_{it}$ contient les instrument de l'individu i au temps t

Donc sur la ligne t de z on aura : $(0,y_{i0},(y_{i0},y_{i1}),y_{i2}..)$ et ce pour les n individus



In [365]:
z=[]
y
#On parcours tout les temps depuis t=2
for i in range(n):
    #On ajoute une ligne avec 2 zeros au début
    z.append([[0]])
    for t in range(2,T):
    #On ajoute dans la ligne i
        instrument=[y[i,t-s] for s in range(2,t+1) ]
        z[i].append(instrument)


Ensuite on définit  $h(\rho,\Delta y_{it},z_{it})$ tel que : $E(h(\rho,\Delta y_{it},z_{it}))=0$.

En l'occurence :  $ h(\rho,\Delta y_{it},z_{it}) = z_{it}(\Delta y_{it}-\rho \Delta y_{it-1})$.




In [366]:
def h(rho,i,t):
    instr=z[i][t]
    return [inst*(delta_y[i,t]-rho*delta_y[i,t-1]) for inst in instr]
h(0.2,2,4)


[-0.0036911403997872826,
 0.0027232167200208325,
 0.01903394608363309,
 0.03955821621342623]

Puis on a : 
$h(\rho,Y_i,Z_i,X_i) = (h(\rho,\Delta y_{11},z_{1T}),h(\rho,\Delta y_{21},z_{2T})...)$

Finalement on minimise le critère : 
    $$Q(\theta)=[\sum_{i=1}^Nh_i(\theta,Y_i)]'W_T[\sum_{i=1}^Nh(\theta,Y_i)]$$

In [367]:
def tab_h(i,rho):
    tab_h=[]
    for t in range(1,T-1):
        tab_h+=h(rho,i,t)
    return np.array(tab_h)

def critere(rho):
    M=1/(n-1)*sum([tab_h(i,rho) for i in range(n-1)])
    return M@M.T


In [368]:
minimize_scalar(critere)

     fun: 0.3241134601299948
    nfev: 9
     nit: 5
 success: True
       x: 0.6835207743977469

In [ ]:
#Matrice qui contiennent yit et yit-1
ysanst0=np.array([yit[i] for i in range(n*T) if (i%T!=0)])
ysanstT=np.array([yit[i] for i in range(n*T) if (i%T!=T-1)])

#Matrice de qui contient les effets fixes 
effetfixe=np.zeros((n*(T-1),n))
for i in range(n):
    for j in range((T-1)*i,(T-1)*i+T-1):
        effetfixe[j][i]=1
effetfixe